In [16]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from gensim import corpora, models
from pprint import pprint


In [17]:
year = [i for i in range(2002, 2021)]
sortedresult = pd.DataFrame()

In [19]:
for i in range(19):
    print("==== "+str(year[i])+" ====")
    data = pd.read_csv("./modi_data/kor_"+str(year[i])+".csv")
    #data = data.drop(['Unnamed: 0'], axis=1)
    
    dfWordList = pd.read_excel("./word_cor.xlsx")
    #print(dfWordList)

    dfWordDel = dfWordList[dfWordList["수정"] == "삭제"] #words to delete
    dfWordMod = dfWordList[dfWordList["수정"] != "삭제"] #words to modify
    #print(dfWordMod)

    seriesDelete = dfWordDel["원본"]
    stopword = []
    for word in seriesDelete.values:
        stopword.append(word)
    #print(listDelete)

    seriesModify = dfWordMod["원본"]
    modiword = []
    for word in seriesModify.values:
        modiword.append(word)
    #print(len(modiword))

    seriesModify2 = dfWordMod["수정"]
    modiword2 = []
    for word in seriesModify2.values:
        modiword2.append(word)
    #print(len(modiword2))


    okt = Okt()
    def oktTokenizer(raw, stopword=stopword, pos=['Noun', 'Alpha']):
        list = []
        for word, tag in okt.pos(raw, #raw data
                                     norm=True, #normalize
                                     stem=True #stemming
                                     ):
            if len(word) > 1 and tag in pos and word not in stopword: 
                if tag == 'Alpha':
                    word = word.lower() #lowercase
                if word in modiword: #for words in modiword
                    word = modiword2[modiword.index(word)] #replace them to modiwords2
                list.append(word)        
        return list

    tokenized = data["kor_full"].apply(lambda row: oktTokenizer(row))
    #print(tokenized)
    #tokenized.to_excel("./finaldata/0911token_full.xls") ##############
    print("========= tokenization completed =========")

    id2word = corpora.Dictionary(tokenized)
    corpus=[id2word.doc2bow(text) for text in tokenized]
    print("# words in total : ", len(id2word))
    print("# documents : ", len(corpus))

    #tfidf
    print("==== calculating tfidf ====")
    tfidf = models.TfidfModel(corpus)

    #tfidf per doc
    tfidflist = []
    for doc in tfidf[corpus]:
        inner_list = [0]*len(id2word) 
        for id, freq in doc:
            inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
        tfidflist.append(inner_list)
    #print(len(tfidflist))

    tfidf_df = pd.DataFrame(tfidflist)
    tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
    #print(tfidf_df)

    total_df = pd.concat([data[["year", "no"]], tfidf_df], axis=1)
    #print(total_df)
    total_df.to_excel("./final_data/tfidf"+str(year[i])+".xlsx") 

    #sum of tfidf for each word
    columnsum = pd.DataFrame(total_df.sum(axis=0)).T
    columnsum = columnsum.drop(['no'], axis=1)
    columnsum['year'] = year[i]
    #print(columnsum)
    columnsum.to_excel("./final_data/sum"+str(year[i])+".xlsx")

    #sort tfidf value in descending order
    columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
    print(columnsum)
    columnsum.to_excel("./final_data/sorted"+str(year[i])+".xlsx")

    print("==== completed ====")

==== 2002 ====
========= tokenization completed =========
# words in total :  370
# documents :  11
==== calculating tfidf ====
   year  컴포넌트    아동   시스템    학습  프로그래밍   콘텐츠    평가    설문   판소리  ...    크기  \
0  2002  0.84  0.72  0.65  0.62   0.61  0.57  0.57  0.56  0.54  ...  0.07   

     네모    대하    독서    등현   디자인    적극    전통   모니터    실행  
0  0.07  0.07  0.07  0.07  0.07  0.07  0.07  0.07  0.07  

[1 rows x 371 columns]
==== completed ====
==== 2003 ====
========= tokenization completed =========
# words in total :  625
# documents :  26
==== calculating tfidf ====
   year    상담    평가    환경    학습    아동  컴포넌트   독서    교수   시스템  ...    정신    편이  \
0  2003  1.46  1.26  1.17  1.11  0.98  0.95  0.9  0.89  0.84  ...  0.06  0.06   

     문헌    학급    허용   idf    대상    먼저    면대    친구  
0  0.06  0.06  0.06  0.06  0.06  0.06  0.06  0.06  

[1 rows x 626 columns]
==== completed ====
==== 2004 ====
========= tokenization completed =========
# words in total :  713
# documents :  30
==== calculating t